In [2]:
# pip install yfinance

In [7]:
import yfinance as yf
from tqdm import tqdm
import pandas as pd

stocks = {
  "Technology": [
    {"name": "Microsoft Corporation", "ticker": "MSFT"},
    {"name": "Apple Inc.", "ticker": "AAPL"},
    {"name": "Alphabet Inc.", "ticker": "GOOGL"},
    {"name": "Intel Corporation", "ticker": "INTC"},
    {"name": "Cisco Systems, Inc.", "ticker": "CSCO"}
  ],
  "Consumer Discretionary": [
    {"name": "Amazon.com, Inc.", "ticker": "AMZN"},
    {"name": "Tesla, Inc.", "ticker": "TSLA"},
    {"name": "The Home Depot, Inc.", "ticker": "HD"},
    {"name": "McDonald's Corporation", "ticker": "MCD"},
    {"name": "Nike, Inc.", "ticker": "NKE"}
  ],
  "Healthcare": [
    {"name": "Johnson & Johnson", "ticker": "JNJ"},
    {"name": "Pfizer Inc.", "ticker": "PFE"},
    {"name": "Merck & Co., Inc.", "ticker": "MRK"},
    {"name": "Abbott Laboratories", "ticker": "ABT"},
    {"name": "Amgen Inc.", "ticker": "AMGN"}
  ],
  "Financials": [
    {"name": "JPMorgan Chase & Co.", "ticker": "JPM"},
    {"name": "Bank of America Corporation", "ticker": "BAC"},
    {"name": "Wells Fargo & Company", "ticker": "WFC"},
    {"name": "Goldman Sachs Group, Inc.", "ticker": "GS"},
    {"name": "Morgan Stanley", "ticker": "MS"}
  ],
  "Industrials": [
    {"name": "Boeing Company", "ticker": "BA"},
    {"name": "Caterpillar Inc.", "ticker": "CAT"},
    {"name": "General Electric Company", "ticker": "GE"},
    {"name": "3M Company", "ticker": "MMM"},
    {"name": "Honeywell International Inc.", "ticker": "HON"}
  ],
  "Consumer Staples": [
    {"name": "The Procter & Gamble Company", "ticker": "PG"},
    {"name": "Coca-Cola Company", "ticker": "KO"},
    {"name": "PepsiCo, Inc.", "ticker": "PEP"},
    {"name": "Walmart Inc.", "ticker": "WMT"},
    {"name": "Costco Wholesale Corporation", "ticker": "COST"}
  ]
}


In [37]:
stocks_list = [st["ticker"]  for key,val in stocks.items() for st in val]
counter = 1
for ticker in tqdm(stocks_list):
    if counter == 1:
        data = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        data["stock"] = [ticker]*len(data)
    else:
        print(ticker)
        df = yf.download(ticker, start="2010-01-01", end="2024-10-03")
        df["stock"] = [ticker]*len(df)
        data = data._append(df, ignore_index=True)
        del df
    counter += 1
        

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


AAPL
GOOGL
INTC
CSCO
AMZN
TSLA
HD


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


MCD
NKE
JNJ
PFE
MRK
ABT


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
 67%|██████▋   | 20/30 [00:00<00:00, 31.79it/s]

AMGN
JPM
BAC
WFC
GS
MS
BA


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


CAT
GE
MMM
HON
PG
KO


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
100%|██████████| 30/30 [00:01<00:00, 28.51it/s]

PEP
WMT
COST


In [38]:
data = data[["stock","Close","Adj Close"]]

,Open,High,Low,Close,Adj Close,Volume,stock
0,30.620001,31.100000,30.590000,30.950001,23.347326,38409100,MSFT
1,30.850000,31.100000,30.639999,30.959999,23.354862,49749600,MSFT
2,30.879999,31.080000,30.520000,30.770000,23.211536,58182400,MSFT
3,30.629999,30.700001,30.190001,30.450001,22.970146,50559700,MSFT
4,30.280001,30.879999,30.240000,30.660000,23.128555,51197400,MSFT
...,...,...,...,...,...,...,...
111233,905.299988,909.239990,898.130005,901.440002,901.440002,2738600,COST
111234,893.260010,893.489990,874.000000,885.619995,885.619995,3459100,COST
111235,891.369995,910.000000,881.409973,886.520020,886.520020,2142900,COST
111236,885.250000,886.989990,872.750000,877.380005,877.380005,1836700,COST
